In [ ]:
import sys
import os
import shutil
from pathlib import Path
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2

In [ ]:
dataset_dir = 'data/'
print(f"Dataset MVTec AD listo en {dataset_dir}")

### Listamos los directorios del dataset

In [ ]:
paths = []
labels = []

for label in glob(str(dataset_dir)+'/*'):
    if 'txt' not in label:
#         print(label)
        paths.append(label+'/*')
        labels.append(label.split('/')[-1])
        
paths

In [ ]:
# Calculamos la cantidad de imagenes en total del dataset
image_paths = glob(str(dataset_dir)+'/*/*/*/*')
print(f"Total de imagenes en el dataset: {len(image_paths)}") 

### Creación de dataframe para mejor análisis

In [ ]:
print(image_paths[0])

In [ ]:
df_mvtec = pd.DataFrame(columns=['label', 'group', 'state', 'filename', 'path'])
dataframes = []
for p in image_paths:
    
    info = p.split('/')
    label = info[1]
    group = info[2]
    state = info[3]
    filename = info[-1]  

    temp_df = pd.DataFrame([{
        'label': label,
        'group': group,
        'state': state,
        'filename': filename,
        'path': p
    }])
    
    dataframes.append(temp_df)

df_mvtec = pd.concat(dataframes, ignore_index=True)

In [ ]:
df_mvtec.head()

## EDA

In [ ]:
# Cantidad de imagenes por clase
df_mvtec[df_mvtec['group'] != 'ground_truth']['label'].value_counts() 

### Grafico de barras de la cantidad de imágenes por clase incluyendo la mascara

In [ ]:
# Contar las imágenes por clase
counts = df_mvtec['label'].value_counts()
classes = counts.index
values = counts.values

# Crear un gráfico de barras
plt.figure(figsize=(15, 6))
colors = sns.color_palette("husl", len(classes))  # Colores diferentes para cada barra

# Crear el gráfico de barras
#bar_plot = sns.barplot(x=classes, y=values, palette=colors)
bar_plot = sns.barplot(x=classes, y=values, hue=classes, palette=colors)

# Agregar la cantidad encima de cada barra
for index, value in enumerate(values):
    bar_plot.text(index, value, str(value), ha='center', va='bottom')

# Etiquetas de los ejes y título
plt.xlabel('Clases de Imágenes', fontsize=14)
plt.ylabel('Cantidad de Imágenes', fontsize=14)
plt.title('Cantidad de Imágenes por Clase', fontsize=16)

# Mostrar el gráfico
plt.xticks(rotation=45)  # Rotar etiquetas del eje x si es necesario
plt.tight_layout()  # Ajustar el layout
plt.show()

### Filtramos para graficar Sin la máscara

In [ ]:
# Filtrar el DataFrame para excluir la mascara
filtered_counts = df_mvtec[df_mvtec["group"] != "ground_truth"]["label"].value_counts()
classes = filtered_counts.index
values = filtered_counts.values

# Crear un gráfico de barras
plt.figure(figsize=(15, 6))
colors = sns.color_palette("husl", len(classes))  # Colores diferentes para cada barra

# Crear el gráfico de barras
bar_plot = sns.barplot(x=classes, y=values, hue=classes, palette=colors)

# Agregar la cantidad encima de cada barra
for index, value in enumerate(values):
    bar_plot.text(index, value, str(value), ha='center', va='bottom')

# Etiquetas de los ejes y título
plt.xlabel('Clases de Imágenes', fontsize=14)
plt.ylabel('Cantidad de Imágenes', fontsize=14)
plt.title('Cantidad de Imágenes por Clase (Excluyendo Ground Truth)', fontsize=16)

# Mostrar el gráfico
plt.xticks(rotation=45)  # Rotar etiquetas del eje x si es necesario
plt.tight_layout()  # Ajustar el layout
plt.show()

### Filtramos para graficar solo el grupo "ground_truth"

In [ ]:
# Filtrar el DataFrame para incluir solo el grupo "ground_truth"
ground_truth_counts = df_mvtec[df_mvtec['group'] == 'ground_truth']['label'].value_counts()
classes = ground_truth_counts.index
values = ground_truth_counts.values

# Crear un gráfico de barras
plt.figure(figsize=(15, 6))
colors = sns.color_palette("husl", len(classes))  # Colores diferentes para cada barra

# Crear el gráfico de barras
bar_plot = sns.barplot(x=classes, y=values, hue=classes, palette=colors)

# Agregar la cantidad encima de cada barra
for index, value in enumerate(values):
    bar_plot.text(index, value, str(value), ha='center', va='bottom')

# Etiquetas de los ejes y título
plt.xlabel('Clases de Imágenes', fontsize=14)
plt.ylabel('Cantidad de Imágenes', fontsize=14)
plt.title('Cantidad de Imágenes por Clase (Ground Truth)', fontsize=16)

# Mostrar el gráfico
plt.xticks(rotation=45)  # Rotar etiquetas del eje x si es necesario
plt.tight_layout()  # Ajustar el layout
plt.show()

Ahora vemos como se distribuyen las imágenes de la mascara

In [ ]:
df_mvtec[df_mvtec['group'] == 'ground_truth']['label'] # Imagenes de Ground Truth o mascara

In [ ]:
# Copiamos el DataFrame original para no modificarlo
madera_df = df_mvtec[df_mvtec['label'] == 'wood'].copy() 
madera_df.head() # Vemos las primeras filas del DataFrame filtrado de madera

In [ ]:
madera_df[madera_df['group'] != 'ground_truth']['state'].value_counts() # Contamos los estados de las imagenes de madera

### Gráficamos la distribución de estados de cada etiqueta o 'label'

In [ ]:
# Iterar sobre cada etiqueta única en el DataFrame
for label in df_mvtec['label'].unique():
    temp_df = df_mvtec[df_mvtec['label'] == label].copy()
    
    plt.figure(figsize=(14, 4))
    
    # Filtrar y contar los estados excluyendo 'ground_truth'
    state_counts = temp_df[temp_df['group'] != 'ground_truth']['state'].value_counts()
    
    # Crear un gráfico de barras
    colors = sns.color_palette("husl", len(state_counts))  # Colores diferentes para cada barra
    barplot = sns.barplot(x=state_counts.index, y=state_counts.values, hue=state_counts.index, palette=colors, legend=False)
    
    # Título del gráfico
    plt.title(f'Cantidad de Imágenes para la Clase: {label}', fontsize=16)
    
    # Agregar la cantidad encima de cada barra
    for p in barplot.patches:
        height = p.get_height()
        barplot.text(x=p.get_x() + p.get_width() / 2, 
                     y=height + 0.05, 
                     s=f'{int(height)}', 
                     ha='center', size=12)
    
    # Etiquetas de los ejes
    plt.xlabel('Estado de las Imágenes', fontsize=14)
    plt.ylabel('Cantidad de Imágenes', fontsize=14)
    
    # Ajustar el layout y mostrar el gráfico
    plt.xticks(rotation=45)  # Rotar etiquetas del eje x si es necesario
    plt.tight_layout()  # Ajustar el layout
    plt.show()

### Agrupamos y generamos imágenes por cada estado de cada etiqueta

In [ ]:
df_sin_mascara = df_mvtec[df_mvtec['group'] != 'ground_truth'] # No incluyo las imágenes de máscara

for label, label_group in df_sin_mascara.groupby('label'):
    for state, state_group in label_group.groupby('state'):
        fig, axs = plt.subplots(1, 5, figsize=(15, 5))
        fig.suptitle(f'{label} - {state}', fontsize=16)
        
        if len(state_group) == 1:
            axs = [axs]  # si solo hay una eje convertir en lista para recorrelo

        for ax, (idx, row) in zip(axs, state_group.iterrows()):
            img_path = row['path']
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            ax.imshow(img)
            ax.axis('off')
            ax.set_title(row['filename'])
            
        fig.tight_layout(pad=1.0)
        fig.subplots_adjust(top=1.15)  # Ajustar espaciado entre títulos y subtramas

        plt.show()
        print('\n\n')

In [ ]:
df_sin_mascara = df_mvtec[df_mvtec['group'] != 'ground_truth'] # No incluyo las imágenes de máscara

for current_class in df_sin_mascara['label'].unique():
    tmp = df_sin_mascara.loc[df_sin_mascara['label'] == current_class]
    states = df_sin_mascara.loc[df_sin_mascara['label'] == current_class, 'state'].unique()

    fig, axs = plt.subplots(1, len(states), figsize=(20, 8))
    
    for col_idx, state in enumerate(states):
        img_dir = tmp.loc[tmp['state'] == state, 'path'].sample(1).iloc[0]
        img = cv2.imread(img_dir)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axs[col_idx].imshow(img)
        axs[col_idx].set_xticks([])  # eliminar las marcas del eje x
        axs[col_idx].set_yticks([])  # eliminar las marcas del eje y

    for ax, col in zip(axs, states):
        ax.set_title(col, size=15)

    axs[0].set_ylabel(current_class, rotation=0, fontsize=15, labelpad=40, fontdict=dict(weight='bold'))

    fig.tight_layout()
    plt.show()
    print('\n\n')